In [1]:
import kfp
from kfp import dsl
import kfp.components as components

In [2]:
%pip install pandas
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 87.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 51.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 103.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 44.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
def gather_data_static(MINIO_SERVER_IP):
    return
    from minio import Minio
    # MINIO_SERVER_IP="172.20.168.171"
    minio_client = Minio(
        f"{MINIO_SERVER_IP}:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )
    file = f"../data/powerconsumption.csv";
    minio_bucket = "mlpipeline"
    minio_client.fput_object(minio_bucket, "powerconsumption.csv", file);

def gather_data_reel(MINIO_SERVER_IP):
    pass

def data_eng(MINIO_SERVER_IP):
    # MINIO_SERVER_IP="172.20.168.171"
    from minio import Minio
    import numpy as np
    import pandas as pd
    import json
    
    minio_client = Minio(
        f"{MINIO_SERVER_IP}:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    );
    minio_bucket = "mlpipeline"
    def store_in_minio(key, src, data, scalers = False):
        from joblib import dump
        if scalers:
            dump(data, src);
        else:
            np.save(src, data);
        minio_client.fput_object(minio_bucket, key, src);

    def normlize(df):
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import MinMaxScaler
        
        df.drop(["DiffuseFlows"], axis=1, inplace=True)
        df.Datetime = pd.to_datetime(df.Datetime);
        reference_datetime = pd.to_datetime("01/01/2017 00:00", format='%d/%m/%Y %H:%M')
        df.Datetime = ((df['Datetime'] - reference_datetime).dt.total_seconds() // 60) % 60
        
        features = ["Datetime", "Temperature", "Humidity", "WindSpeed", "GeneralDiffuseFlows"]
        target = ['PowerConsumption_Zone1']
        X = df[features]
        y = df[target]
        
        # Normalize the features and target using Min-Max scaling
        scaler_X = MinMaxScaler()
        X_scaled = scaler_X.fit_transform(X)

        scaler_y = MinMaxScaler()
        y_scaled = scaler_y.fit_transform(y)

        # Define the number of time steps ( 1 day)
        time_steps = 6*24
        
        # Create sequences for LSTM input
        X_sequences = []
        y_sequences = []
        
        for i in range(len(X_scaled) - time_steps + 1):
            X_sequences.append(X_scaled[i:i+time_steps, :])
            # Use the avg of the last consumption as target of the last 24 hours
            y_sequences.append(np.mean(y_scaled[i:i+time_steps, 0]))
        X_sequences = np.array(X_sequences)
        y_sequences = np.array(y_sequences)
        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, # for xgboost 
            y_scaled,  # for xgboost
            # X_sequences,  # for lstm
            # y_sequences,  # for lstm
            test_size=0.4,
            random_state=42
        )
        
        
        X_cv_test = X_test[:len(X_test)//2]
        X_test = X_test[len(X_test)//2:]
        y_cv_test = y_test[:len(y_test)//2]
        y_test = y_test[len(y_test)//2:]
        
        return X_train, X_cv_test, X_test, y_train, y_cv_test, y_test, scaler_X, scaler_y 
        

    def fetch(data):
        file = f"/tmp/{data}";
        minio_client.fget_object(minio_bucket, data, file);
        return pd.read_csv(file);
        
        
    #fetch data
    df = fetch("powerconsumption.csv");
    
    # after fetching normlize the data
    X_train, X_cv_test, X_test, y_train, y_cv_test, y_test, scaler_X, scaler_y = normlize(df);
    
    # store each of the training, testing and cv dataset in minio
    store_in_minio("norml/X_train.npy", "/tmp/X_train.npy", X_train)
    store_in_minio("norml/y_train.npy", "/tmp/y_train.npy", y_train)
    store_in_minio("norml/X_test.npy", "/tmp/X_test.npy", X_test)
    store_in_minio("norml/y_test.npy", "/tmp/y_test.npy", y_test)
    store_in_minio("norml/X_cv_test.npy", "/tmp/X_cv_test.npy", X_cv_test)
    store_in_minio("norml/y_cv_test.npy", "/tmp/y_cv_test.npy", y_cv_test)
    store_in_minio("norml/scaler_X.npy", "/tmp/scaler_X.npy", scaler_X, scalers = True)
    store_in_minio("norml/scaler_y.npy", "/tmp/scaler_y.npy", scaler_y, scalers = True)
    # done

    
def unit_testing(MINIO_SERVER_IP):
    """
    check for the model code base
    """
    print("[+] Tests");


def building_lstm(MINIO_SERVER_IP):
    """
    build Models
    """
    # MINIO_SERVER_IP="172.20.168.171"
    from minio import Minio
    import numpy as np
    import tensorflow as tf
    import os
    import glob
        
    minio_client = Minio(
        f"{MINIO_SERVER_IP}:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    );
    minio_bucket = "mlpipeline"
    
    def fetch_from_minio(key, dst, scalers = False):
        from joblib import load
        
        minio_client.fget_object(minio_bucket, key, dst);
        if scalers:
            return load(dst);
        else:
            return np.load(dst);
    
    def store_in_minio(key, src, data, scalers = False):
        from joblib import dump
        if scalers:
            dump(data, src);
        else:
            np.save(src, data);
        minio_client.fput_object(minio_bucket, key, src);
    
    def upload_local_directory_to_minio(local_path, bucket_name, minio_path):
        assert os.path.isdir(local_path)
        for local_file in glob.glob(local_path + '/**'):
            local_file = local_file.replace(os.sep, "/") # Replace \ with / on Windows
            if not os.path.isfile(local_file):
                upload_local_directory_to_minio(
                    local_file, bucket_name, minio_path + "/" + os.path.basename(local_file))
            else:
                remote_path = os.path.join(
                    minio_path, local_file[1 + len(local_path):])
                remote_path = remote_path.replace(
                    os.sep, "/")  # Replace \ with / on Windows
                minio_client.fput_object(bucket_name, remote_path, local_file)
                
    # this should global variable
    time_steps = 6*24
    features = ["Datetime", "Temperature", "Humidity", "WindSpeed", "GeneralDiffuseFlows"]
    target = ['PowerConsumption_Zone1']
    
    # build
    X_train = fetch_from_minio("norml/X_train.npy", "/tmp/X_train.npy")
    y_train = fetch_from_minio("norml/y_train.npy", "/tmp/y_train.npy")
    
    print(X_train.shape, y_train.shape)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(16, activation='tanh', stateful=True, return_sequences=True, batch_input_shape=(1, time_steps, len(features))),
        #tf.keras.layers.LSTM(16, activation='tanh', stateful=True, return_sequences=True, batch_input_shape=(1, (31363 ,time_steps, len(features)))),
        #tf.keras.layers.LSTM(16, activation='tanh', input_shape=(time_steps, len(features))),
        tf.keras.layers.Dense(1, activation='linear')
    ]);
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error');
    #X_train.reshape(X_train.shape[0]//time_steps , time_steps, len(features))
    # model.fit(X_train, y_train)
    
    model_path = "/tmp/lstm.model"
    minio_path = "models/lstm/1/"
    model.save(model_path);
    upload_local_directory_to_minio(model_path, "mlpipeline", minio_path)

def building_XGBoost(MINIO_SERVER_IP):
    """
    build Models
    """
    # MINIO_SERVER_IP="172.20.168.171"
    from minio import Minio
    import numpy as np
    import tensorflow as tf
    import os
    import glob
    from sklearn.ensemble import RandomForestRegressor
    import pickle
        
    minio_client = Minio(
        f"{MINIO_SERVER_IP}:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    );
    minio_bucket = "mlpipeline"
    
    def store_in_minio(key, src):
        minio_client.fput_object(minio_bucket, key, src);
    
    # this should global variable
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)    
    model_path = "/tmp/xgboost.model"
    minio_path = "models/xgboost/1/xg.model"
    with open(model_path, "wb") as f: pickle.dump(rf_model, f);
    store_in_minio(minio_path, model_path)
    

def testing(MINIO_SERVER_IP):
    """
    testing
    """
    print("[+] Tests");

    
def hyper_tunning_lstm(MINIO_SERVER_IP):
    """
    hyper the models
    """
    
    
def hyper_tunning_XGBoost(MINIO_SERVER_IP):
    """
    store the models
    """
    # MINIO_SERVER_IP="172.20.168.171"
    from minio import Minio
    import numpy as np
    import tensorflow as tf
    import os
    import glob
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    import pickle
        
    minio_client = Minio(
        f"{MINIO_SERVER_IP}:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    );
    minio_bucket = "mlpipeline"
    
    def fetch_from_minio(key, dst, scalers = False):
        from joblib import load
        
        minio_client.fget_object(minio_bucket, key, dst);
        if scalers:
            return load(dst);
        else:
            return np.load(dst);
    def fetch_model(key, dst):
        model = minio_client.fget_object(minio_bucket, key, dst);
        with open(dst, 'rb') as mod:
            return pickle.load(mod)
    def store_in_minio(key, src):
        minio_client.fput_object(minio_bucket, key, src);
    
    # get the data
    X_train = fetch_from_minio("norml/X_train.npy", "/tmp/X_train.npy")
    y_train = fetch_from_minio("norml/y_train.npy", "/tmp/y_train.npy")
    X_cv_test = fetch_from_minio("norml/X_cv_test.npy", "/tmp/X_cv_test.npy")
    y_cv_test = fetch_from_minio("norml/y_cv_test.npy", "/tmp/y_cv_test.npy")
    scaler_X = fetch_from_minio("norml/scaler_X.npy", "/tmp/scaler_X.npy", scalers = True)
    scaler_y = fetch_from_minio("norml/scaler_y.npy", "/tmp/scaler_y.npy", scalers = True)
    
    # get the model
    model_path = "/tmp/xgboost.model"
    minio_path = "models/xgboost/1/xg.model"
    rf_model = fetch_model(minio_path, model_path);
    # fit the model
    rf_model.fit(X_train, y_train.ravel())  # Ravel y_train to convert it to a 1D array
    # hyper parametrs
    # Make predictions using the test dataset
    y_pred = rf_model.predict(X_cv_test)

    # Inverse transform the scaled predictions to the original scale
    y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

    # Inverse transform the scaled actual values (if needed)
    y_actual_original = scaler_y.inverse_transform(y_cv_test.reshape(-1, 1))
    mse = mean_squared_error(y_actual_original, y_pred_original)
    print(f"Mean Squared Error: {mse:.2f}")
    
    
    # store the new trained model
    with open(model_path, "wb") as f: pickle.dump(rf_model, f);
    store_in_minio(minio_path, model_path);
    
def automl(MINIO_SERVER_IP):
    """
    best model
    """
    pass

def serving(MINIO_SERVER_IP):
    """
    Create kserve instance
    """
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1TFServingSpec
    from datetime import datetime

    namespace = utils.get_default_target_namespace()
    # MINIO_SERVER_IP="172.20.168.171"
    

    now = datetime.now()
    v = now.strftime("%Y-%m-%d--%H-%M-%S")

    name='inference-service-{}'.format(v)
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version
    
    # best_model = f"http://{MINIO_SERVER_IP}:9000/mlpipeline/models/xgboost"
    best_model = f"http://{MINIO_SERVER_IP}:9000/mlpipeline/models/lstm"
    isvc = V1beta1InferenceService(
        api_version=api_version,
        kind=constants.KSERVE_KIND,
        metadata=client.V1ObjectMeta(
            name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}
        ),
        spec=V1beta1InferenceServiceSpec(
            predictor=V1beta1PredictorSpec(
                service_account_name="kserver-access",
                tensorflow=(
                    V1beta1TFServingSpec(storage_uri="s3://mlpipeline/models/lstm")
                )
            )
        )
    )

    KServe = KServeClient()
    KServe.create(isvc)

    
MINIO = "172.20.95.46"
# data_eng()
# building_XGBoost()
# building_lstm()
# hyper_tunning_XGBoost()
# serving()
#gather_data_static(MINIO)

In [34]:
# base_img = "public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0"
base_img = "public.ecr.aws/kubeflow-on-aws/notebook-servers/jupyter-tensorflow:2.12.0-cpu-py310-ubuntu20.04-ec2-v1.0"

comp_gather_data_static = components.create_component_from_func(
    gather_data_static,
    # base_image = base_img,
    # packages_to_install=['pandas', 'scikit-learn']
);
comp_gather_data_reel = components.create_component_from_func(
    gather_data_reel,
    # base_image = base_img,
    # packages_to_install=['pandas', 'scikit-learn']
);
comp_data_eng = components.create_component_from_func(
    data_eng,
    base_image = base_img,
    packages_to_install=['pandas', 'scikit-learn']
);
comp_unit_testing = components.create_component_from_func(
    unit_testing,
    # base_image = base_img,
    # packages_to_install=['pandas', 'scikit-learn']
);
comp_building_lstm = components.create_component_from_func(
    building_lstm,
    base_image = base_img,
    packages_to_install=['joblib']
);
comp_building_XGBoost = components.create_component_from_func(
    building_XGBoost,
    base_image = base_img,
    packages_to_install=['scikit-learn']
);
comp_testing = components.create_component_from_func(
    testing,
    # base_image = base_img,
    # packages_to_install=['pandas', 'scikit-learn']
);
comp_hyper_tunning_lstm = components.create_component_from_func(
    hyper_tunning_lstm,
    # base_image = base_img,
    # packages_to_install=['pandas', 'scikit-learn']
);
comp_hyper_tunning_XGBoost = components.create_component_from_func(
    hyper_tunning_XGBoost,
    base_image = base_img,
    packages_to_install=['scikit-learn']
);
comp_automl = components.create_component_from_func(
    automl,
    # base_image = base_img,
    # packages_to_install=['pandas', 'scikit-learn']
);
comp_serving = components.create_component_from_func(
    serving,
    base_image = base_img,
    packages_to_install=['kserve']
);

In [35]:
@dsl.pipeline(name="power-consumption", description="predict power")
def output(no_epochs, optimizer, MINIO_SERVER_IP):
    step1_1 = comp_gather_data_static(MINIO_SERVER_IP);
    step1_2 = comp_gather_data_reel(MINIO_SERVER_IP);
    
    step2 = comp_data_eng(MINIO_SERVER_IP);
    step2.after(step1_1);
    step2.after(step1_2);
    
    step3 = comp_unit_testing(MINIO_SERVER_IP);
    step3.after(step2)
    
    step4_1 = comp_building_lstm(MINIO_SERVER_IP);
    step4_2 = comp_building_XGBoost(MINIO_SERVER_IP); 
    step4_1.after(step3);
    step4_2.after(step3);
    
    step5 = comp_testing(MINIO_SERVER_IP);
    step5.after(step4_1);
    step5.after(step4_2);
    
    step6_1 = comp_hyper_tunning_lstm(MINIO_SERVER_IP);
    step6_2 = comp_hyper_tunning_XGBoost(MINIO_SERVER_IP);
    step6_1.after(step5);
    step6_2.after(step5);
    
    step7 = comp_automl(MINIO_SERVER_IP);
    step7.after(step6_1);
    step7.after(step6_2);
    
    step8 = comp_serving(MINIO_SERVER_IP);
    step8.after(step7);

In [36]:
client = kfp.Client()
arguments = {
    "no_epochs" : 1,
    "optimizer": "adam",
    "MINIO_SERVER_IP": "172.20.95.46"
}
    
client.create_run_from_pipeline_func(output, arguments=arguments,experiment_name="my_testing")

RunPipelineResult(run_id=0ced5695-bc6d-406d-a879-391c9c527978)

In [11]:
kfp.compiler.Compiler().compile(pipeline_func=output,package_path='output.yaml')

pipeline_id = "c09189ac-7763-43e4-8397-0f3dd8278261"
pipelines = client.list_pipelines()
for pipeline in pipelines.pipelines:
    print(pipeline.id)
    #if pipeline.name == "pipelines1":
    #    pipeline_id = pipeline.id
    #    print("found")
    #    break

client.upload_pipeline_version(
    pipeline_package_path='output.yaml',
    pipeline_version_name="0.4",
    pipeline_id = pipeline_id,
    # pipeline_name="pipelines1",
    description="just for testing"
)

7e135877-a255-46ec-9459-c30dd038c68e
107961eb-cdef-495f-b649-13f0603f45c3
6e1cd39f-d561-48db-ad92-bfd6f2ed4653
48f88cf0-f2be-474d-91f9-fe9e3f93818b
e3c5a5cc-f4e1-42b4-8045-cc3cc4df516c


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'date': 'Thu, 07 Sep 2023 03:11:42 GMT', 'content-length': '365', 'content-type': 'text/plain; charset=utf-8', 'x-envoy-upstream-service-time': '13', 'server': 'envoy'})
HTTP response body: {"error_message":"Failed to get namespace from pipelineId.: Failed to get namespace from Pipeline ID: ResourceNotFoundError: Pipeline c09189ac-7763-43e4-8397-0f3dd8278261 not found.","error_details":"Failed to get namespace from pipelineId.: Failed to get namespace from Pipeline ID: ResourceNotFoundError: Pipeline c09189ac-7763-43e4-8397-0f3dd8278261 not found."}


In [103]:
MINIO_SERVER_IP="172.20.168.171"
from minio import Minio
import numpy as np
import pandas as pd
import json
from joblib import load
minio_client = Minio(
    f"{MINIO_SERVER_IP}:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
);
minio_bucket = "mlpipeline"
data = "norml/scaler_X.npy"
file = f"/tmp/{data}";
minio_client.fget_object(minio_bucket, data, file);
obj = load(file)

[[  50.      76.773  261.72    25.782 5814.984]]


In [86]:
%pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=281859 sha256=f0920d07f83cb4ffb2d3b3297d1840c64d8e087e21b5d5472218b9dc07a92ea9
  Stored in directory: /home/jovyan/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
